In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 10


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-07 14:00:53] ax.service.ax_client: Generated new trial 160 with parameters {'Drug': 92, 'SL_1': 71, 'SL_2': 0, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 2, 'S_2': 0, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the re

10 min


In [7]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,2,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,1,0,0
4,0,0,3,0
5,0,0,2,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [8]:
mask = (unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']] == 0).all(axis=1)

# print the rows that are about to change
print("Rows to be updated (before):")
print(unlabeled_trial.loc[mask])

# set Water to 100 on those rows
unlabeled_trial.loc[mask, 'Water'] = 100

# print the rows after the update
print("\nRows after update:")
print(unlabeled_trial.loc[mask])

Rows to be updated (before):
    trial_index  Drug  SL_1  SL_2  SL_3  LL_1  LL_2  LL_3  P_1  P_2  ...  S_1  \
1           161    89    66     0     0     0     0     0    0    0  ...    0   
2           162    89    67     0     0     0     0     0    0    0  ...    0   
6           166    89    67     0     0     0     0     0    0    0  ...    0   
7           167    89    67     0     0     0     0     0    0    0  ...    0   
8           168    99    74     0     0     0     0     0    0    0  ...    0   
9           169    99    74     0     0     0     0     0    0    0  ...    0   
10          170    89    67     0     0     0     0     0    0    0  ...    0   
11          171    99    74     0     0     0     0     0    0    0  ...    0   
12          172    99    74     0     0     0     0     0    0    0  ...    0   
13          173    99    74     0     0     0     0     0    0    0  ...    0   
14          174    89    67     0     0     0     0     0    0    0  ...    0   

In [9]:
unlabeled_trial[['S_1', 'S_2', 'S_3', 'Water']]

,S_1,S_2,S_3,Water
0,2,0,0,0
1,0,0,0,100
2,0,0,0,100
3,0,1,0,0
4,0,0,3,0
5,0,0,2,0
6,0,0,0,100
7,0,0,0,100
8,0,0,0,100
9,0,0,0,100


In [10]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0000,16.0000,16.000000,16.000000
mean,167.500000,0.569125,0.430875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0625,0.125000,0.750000
std,4.760952,0.006742,0.006742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2500,0.2500,0.341565,0.447214
min,160.000000,0.546000,0.426000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.000000,0.000000
25%,163.750000,0.571000,0.428000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.000000,0.750000
50%,167.500000,0.571000,0.429000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.000000,1.000000
75%,171.250000,0.572000,0.429000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.000000,1.000000
max,175.000000,0.574000,0.454000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,1.0000,1.000000,1.000000


In [11]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000,16.0000,16.000000,16.00000
mean,167.500000,93.125000,70.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0625,0.312500,75.00000
std,4.760952,4.801042,3.464102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.2500,0.873212,44.72136
min,160.000000,89.000000,66.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.000000,0.00000
25%,163.750000,89.000000,67.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.000000,75.00000
50%,167.500000,90.500000,71.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.000000,100.00000
75%,171.250000,99.000000,74.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0000,0.000000,100.00000
max,175.000000,99.000000,74.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000,1.0000,3.000000,100.00000


In [12]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.000000,16.0
mean,167.500000,199.193750,150.806250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.5,62.5,125.000000,750.000000,0.0
std,4.760952,2.359582,2.359582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,341.565026,447.213595,0.0
min,160.000000,191.100000,149.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
25%,163.750000,199.850000,149.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,750.000000,0.0
50%,167.500000,199.850000,150.150000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1000.000000,0.0
75%,171.250000,200.200000,150.150000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1000.000000,0.0
max,175.000000,200.900000,158.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,1000.0,1000.000000,1000.000000,0.0


In [13]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        3.2
SL_1        2.4
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.0
LL_3        0.0
P_1         0.0
P_2         0.0
P_3         0.0
S_1         1.0
S_2         1.0
S_3         2.0
Water      12.0
Solvent     0.0
dtype: float64

In [14]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [15]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [16]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [17]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [18]:
updated_script

('protocol/iteration_10_OT_2_protocol_1.py',
 'protocol/iteration_10_OT_2_protocol_2.py')

# Perform experiment

# Results

In [20]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,160,0.564,0.436,0,0,0,0,0,0,0,0,1,0,0,0,2,0
1,161,0.574,0.426,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,162,0.571,0.429,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,163,0.571,0.429,0,0,0,0,0,0,0,0,0,1,0,0,2,0
4,164,0.564,0.436,0,0,0,0,0,0,0,0,0,0,1,0,2,0
5,165,0.546,0.454,0,0,0,0,0,0,0,0,0,0,1,0,2,0
6,166,0.571,0.429,0,0,0,0,0,0,0,0,0,0,0,1,1,0
7,167,0.571,0.429,0,0,0,0,0,0,0,0,0,0,0,1,1,0
8,168,0.572,0.428,0,0,0,0,0,0,0,0,0,0,0,1,1,0
9,169,0.572,0.428,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [21]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,296.312330,0.188044
1,Good,A2,283.137940,0.148372
2,Good,A3,292.999530,0.215014
3,Caution,A4,272.088903,0.340801
4,Good,A5,250.030673,0.211513
5,Caution,A6,282.663084,0.245079
6,Caution,B1,251.239377,0.431136
7,Good,B2,248.814415,0.232086
8,Good,B3,250.127926,0.171242
9,Caution,B4,131.796476,0.570796


In [22]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,290.816600,6.853104,0.183810,0.033522,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,249.471171,0.928792,0.201664,0.043023,1
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,234.749973,14.885378,0.383012,0.021149,1
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,239.970798,1.032257,0.220637,0.040480,1
7,Formulation 8,241.083325,5.831588,0.257343,0.081815,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,243.063254,6.223093,0.211898,0.078326,1


In [23]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,160,0.564,0.436,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 1,290.816600,6.853104,0.183810,0.033522,1
1,161,0.574,0.426,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,162,0.571,0.429,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 3,249.471171,0.928792,0.201664,0.043023,1
3,163,0.571,0.429,0,0,0,0,0,0,0,...,0,0,2,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,164,0.564,0.436,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 5,234.749973,14.885378,0.383012,0.021149,1
5,165,0.546,0.454,0,0,0,0,0,0,0,...,1,0,2,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,166,0.571,0.429,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 7,239.970798,1.032257,0.220637,0.040480,1
7,167,0.571,0.429,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 8,241.083325,5.831588,0.257343,0.081815,1
8,168,0.572,0.428,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,169,0.572,0.428,0,0,0,0,0,0,0,...,0,1,1,0,Formulation 10,243.063254,6.223093,0.211898,0.078326,1


In [24]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,160,0.564,0.436,0,0,0,0,0,0,0,...,2,0,Formulation 1,290.816600,6.853104,0.183810,0.033522,1,1128.0,0
1,161,0.574,0.426,0,0,0,0,0,0,0,...,1,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,162,0.571,0.429,0,0,0,0,0,0,0,...,1,0,Formulation 3,249.471171,0.928792,0.201664,0.043023,1,1142.0,0
3,163,0.571,0.429,0,0,0,0,0,0,0,...,2,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,164,0.564,0.436,0,0,0,0,0,0,0,...,2,0,Formulation 5,234.749973,14.885378,0.383012,0.021149,1,1128.0,0
5,165,0.546,0.454,0,0,0,0,0,0,0,...,2,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,166,0.571,0.429,0,0,0,0,0,0,0,...,1,0,Formulation 7,239.970798,1.032257,0.220637,0.040480,1,1142.0,0
7,167,0.571,0.429,0,0,0,0,0,0,0,...,1,0,Formulation 8,241.083325,5.831588,0.257343,0.081815,1,1142.0,0
8,168,0.572,0.428,0,0,0,0,0,0,0,...,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,169,0.572,0.428,0,0,0,0,0,0,0,...,1,0,Formulation 10,243.063254,6.223093,0.211898,0.078326,1,1144.0,0


In [25]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.290817,0.006853,0.564,0.0,0.183810,0.033522,0.166667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
2,1,0.249471,0.000929,0.571,0.0,0.201664,0.043023,0.083333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
4,1,0.234750,0.014885,0.564,0.0,0.383012,0.021149,0.166667
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
6,1,0.239971,0.001032,0.571,0.0,0.220637,0.040480,0.083333
7,1,0.241083,0.005832,0.571,0.0,0.257343,0.081815,0.083333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,1,0.243063,0.006223,0.572,0.0,0.211898,0.078326,0.083333


In [26]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.290817,0.006853,0.564,0.0,0.183810,0.033522,0.166667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
2,1,0.249471,0.000929,0.571,0.0,0.201664,0.043023,0.083333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
4,1,0.234750,0.014885,0.564,0.0,0.383012,0.021149,0.166667
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
6,1,0.239971,0.001032,0.571,0.0,0.220637,0.040480,0.083333
7,1,0.241083,0.005832,0.571,0.0,0.257343,0.081815,0.083333
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,1,0.243063,0.006223,0.572,0.0,0.211898,0.078326,0.083333


In [27]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [28]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [29]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,160,0.564,0.436,0,0,0,0,0,0,0,...,0.166667,0,Formulation 1,0.290817,0.006853,0.183810,0.033522,1,0.564,0
1,161,0.574,0.426,0,0,0,0,0,0,0,...,0.083333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,162,0.571,0.429,0,0,0,0,0,0,0,...,0.083333,0,Formulation 3,0.249471,0.000929,0.201664,0.043023,1,0.571,0
3,163,0.571,0.429,0,0,0,0,0,0,0,...,0.166667,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,164,0.564,0.436,0,0,0,0,0,0,0,...,0.166667,0,Formulation 5,0.234750,0.014885,0.383012,0.021149,1,0.564,0


In [30]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [31]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [32]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 160 with data: {'Solu': (0.564, 0), 'Size': (0.290817, 0.006853), 'PDI': (0.18381, 0.033522)}.
[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 161 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 162 with data: {'Solu': (0.571, 0), 'Size': (0.249471, 0.000929), 'PDI': (0.201664, 0.043023)}.
[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 163 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 164 with data: {'Solu': (0.564, 0), 'Size': (0.23475, 0.014885), 'PDI': (0.383012, 0.021149)}.
[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 165 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-08 07:59:27] ax.service.ax_client: Completed trial 166 with data: {'Solu': (0.571, 0), 'Size': (0.239971, 0.001032), 'PDI':